In [1]:
from sam_models import *
from sam_orm_queries import create_sam_engine, get_session, find_user_by_username, get_project_with_full_details

engine, SessionLocal = create_sam_engine()

hpc-reader:$SAM_DB_PASSWORD@sam-sql.ucar.edu/sam


In [2]:
with get_session(SessionLocal) as session:
    # Find a user
    user = find_user_by_username(session, 'benkirk')
    if user:
        print(f"Found user: {user.full_name}")
        print(f"Primary GID: {user.primary_gid}")
        print(f"Primary email: {user.primary_email}")
        print(f"All emails: {', '.join(user.all_emails)}")

        # Get detailed email info
        print("Detailed email information:")
        for email_info in user.get_emails_detailed():
            primary_marker = " (PRIMARY)" if email_info['is_primary'] else ""
            active_marker = "" if email_info['active'] else " (INACTIVE)"
            print(f"  - {email_info['email']}{primary_marker}{active_marker}")

        # Find projects
        print("Detailed project information:")
        for p in user.all_projects:
            label = "" if p.active else "** INACTIVE **"
            print(f"  {p.projcode}, {p.title}{label}")


Found user: Benjamin Shelton Kirk
Primary GID: 1000
Primary email: benkirk@ucar.edu
All emails: benkirk@ucar.edu
Detailed email information:
  - benkirk@ucar.edu
Detailed project information:
  SCSG0001, CSG systems project
  NCGD0058, Software Engineering At Scale (SEAS)** INACTIVE **
  UESM0011, 2022 CESM tutorial August 8-12** INACTIVE **
  UCIS0004, GPU Training Allocation for Workshop** INACTIVE **


In [3]:
def project_details(project):
    if project:
        print(f"\n--- Project Details ---")
        print(f"Project: {project.projcode}")
        print(f"Title: {project.title}")
        print(f"Lead: {project.lead.full_name}")
        for d in project.directories:
            label = "" if d.is_currently_active else " ** INACTIVE **"
            print(f"  Directory: {d.directory_name}{label}")
        # Show allocations by resource
        print(f"Allocations by resource:")
        allocs_by_resource = project.get_all_allocations_by_resource()
        for resource_name, alloc in allocs_by_resource.items():
            print(f"  {resource_name:12}: {alloc.amount:,.2f} (expires {alloc.end_date})")

        # Show users on project
        print(f"Users:")
        for user in project.users:
            print(f"  {user.username:12} {user.display_name:30} <{user.primary_email}>")


In [4]:
with get_session(SessionLocal) as session:
    # Find a project
    project = get_project_with_full_details(session, 'SCSG0001')
    project_details(project)


--- Project Details ---
Project: SCSG0001
Title: CSG systems project
Lead: Benjamin Shelton Kirk
  Directory: /glade/p/CSG ** INACTIVE **
  Directory: /gpfs/csfs1/cisl/csg
  Directory: /glade/p/cisl/CSG
  Directory: /stratus/USS
Allocations by resource:
  GLADE       : 10.00 (expires 2026-09-30 23:59:59)
  Data_Access : 1.00 (expires 2026-09-30 23:59:59)
  HPC_Futures_Lab: 1.00 (expires 2026-09-30 23:59:59)
  Campaign_Store: 20.00 (expires 2026-09-30 23:59:59)
  Casper      : 50,000.00 (expires 2026-09-30 23:59:59)
  Stratus     : 100.00 (expires 2026-09-30 23:59:59)
  Casper GPU  : 10,000.00 (expires 2026-09-30 23:59:59)
  Derecho     : 100,000,000.00 (expires 2026-09-30 23:59:59)
  Derecho GPU : 1,000,000.00 (expires 2026-09-30 23:59:59)
  Gust        : 5,000.00 (expires 2026-09-30 23:59:59)
  Gust GPU    : 1,000.00 (expires 2026-09-30 23:59:59)
Users:
  benkirk      Ben Kirk                       <benkirk@ucar.edu>
  csgteam      Ben Kirk                       <None>
  rory        

In [5]:
from sam_orm_queries import get_projects_expiring_soon
with get_session(SessionLocal) as session:
    # Get expiring projects (simple) - all resources
    print("\n--- Projects Expiring Soon (30 days, all resources) ---")
    expiring = get_projects_expiring_soon(session, days=30)
    print(f"Found {len(expiring)} allocations expiring")
    for proj, alloc, res_name, days in expiring:
        print(f"  {proj.projcode} / {days} days remaining / ({res_name})")
 




--- Projects Expiring Soon (30 days, all resources) ---
Found 42 allocations expiring
  UCLA0033 / 28 days remaining / (Derecho)
  UGMU0031 / 28 days remaining / (Derecho)
  UDEL0008 / 28 days remaining / (Casper GPU)
  UWAS0104 / 28 days remaining / (Casper)
  UPSU0045 / 28 days remaining / (Derecho)
  UCSU0092 / 28 days remaining / (Derecho)
  UUSL0030 / 28 days remaining / (Casper GPU)
  UCUB0115 / 28 days remaining / (Derecho)
  UCIR0047 / 28 days remaining / (Campaign_Store)
  UWAS0113 / 28 days remaining / (Campaign_Store)
  UWAS0115 / 28 days remaining / (Derecho)
  UPSU0046 / 28 days remaining / (Derecho)
  UCSU0106 / 28 days remaining / (Derecho)
  UPRI0023 / 28 days remaining / (Derecho)
  WYOM0168 / 28 days remaining / (Casper GPU)
  UUTA0012 / 28 days remaining / (Campaign_Store)
  UWAS0138 / 28 days remaining / (Campaign_Store)
  UCLA0063 / 28 days remaining / (Campaign_Store)
  NHAO0031 / 28 days remaining / (Data_Access)
  UAUB0012 / 28 days remaining / (Casper)
  UHAR0

In [9]:
from sam_models_jobs_queries import *

projcode='SCSG0001'
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 12, 31)
resource='Casper'

with get_session(SessionLocal) as session:

    top_users = get_user_usage_on_project(        
                    session, projcode,
                    start_date,
                    end_date,
                    limit=5)
    for user in top_users:
        print(f"{user['username']:12}: {user['charges']:.2f}")
                
    usage = get_project_usage_summary(session,
                                      projcode,
                                      start_date,
                                      end_date,
                                      resource)
    #print(usage)
    print(f"Project {projcode} ran {usage['total_jobs']} and used {usage['total_core_hours']:2f} core hours")

    trend = get_daily_usage_trend(session,
                                  projcode,
                                  start_date,
                                  end_date,
                                  resource)
    for day in trend:
        print(f"{day['date']}: {day['jobs']} jobs, {day['charges']} charges")

benkirk     : 2120606.22
negins      : 1293099.06
csgteam     : 1174065.03
rory        : 1048022.02
bneuman     : 12598.27


TypeError: unsupported operand type(s) for /: 'float' and 'decimal.Decimal'